In [1]:
import os
import getpass


def _set_env(name: str):
    if not os.getenv(name):
        os.environ[name] = getpass.getpass(f"{name}: ")


_set_env("GOOGLE_API_KEY")

In [8]:
import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
!python -m pip install -qU langchain_anthropic

In [3]:
import operator
from typing import Annotated, TypedDict
from langchain_core.pydantic_v1 import BaseModel
#from pydantic import BaseModel # Does not work with Pydantic V2
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_ollama import ChatOllama
from langgraph.constants import Send
from langgraph.graph import END, StateGraph, START
from langchain_mistralai import ChatMistralAI
from langchain_anthropic import ChatAnthropic

# Define the model
model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash-latest") #"gemini-1.5-pro-exp-0827")
#model = ChatGoogleGenerativeAI(model= "gemini-1.5-pro-exp-0827") - better results but quota exhaustion

#model = ChatOllama(model="llama3.1:8b")

#model = ChatMistralAI(model="mistral-large-latest") #"open-mistral-7b")

#model = ChatAnthropic(model="claude-3-5-sonnet-20240620")
# Define the prompts
step1_prompt = """Step 1: I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}"""

step2_prompt = """Step 2: For each of the three proposed solutions in Step1, 
Review their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors.

Solutions:
{solutions}"""

step3_prompt = """Step 3: For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

Evaluation:
{review}"""

step4_prompt = """Step 4: Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution.

Detailed analysis:
{deepen_thought_process}"""

# Define the output structures
class Solutions(BaseModel):
    """Solutions Provided"""
    solutions: list[str]

class Review(BaseModel):
    """Reviews on Solutions"""
    review: str

class DeepThought(BaseModel):
    """Deep Thoughts on Reviews"""
    deep_thought: str

class RankedSolutions(BaseModel):
    """Ranking of Solutions"""
    ranked_solutions: str

# Define the overall state
class OverallState(TypedDict):
    """Overall State Flow"""
    input: str
    perfect_factors: str
    solutions: Annotated[list[str], operator.add]
    reviews: Annotated[list[str], operator.add]
    deep_thoughts: Annotated[list[str], operator.add]
    ranked_solutions: str

# Define the state for individual solution processing
class SolutionState(TypedDict):
    """Individual Processiing of SolutionState"""
    solution: str

c:\Users\dpokh\langchain-academy\lc-academy-env\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Graph components
def generate_solutions(state: OverallState):
    prompt = step1_prompt.format(input=state["input"], perfect_factors=state["perfect_factors"])
    response = model.with_structured_output(Solutions).invoke(prompt)
    return {"solutions": response.solutions}

def evaluate_solution(state: SolutionState):
    prompt = step2_prompt.format(solutions=state["solution"])
    response = model.with_structured_output(Review).invoke(prompt)
    return {"reviews": [response.review]}

def deepen_thought(state: SolutionState):
    prompt = step3_prompt.format(review=state["solution"])
    response = model.with_structured_output(DeepThought).invoke(prompt)
    return {"deep_thoughts": [response.deep_thought]}

def rank_solutions(state: OverallState):
    deep_thoughts = "\n\n".join(state["deep_thoughts"])
    prompt = step4_prompt.format(deepen_thought_process=deep_thoughts)
    response = model.with_structured_output(RankedSolutions).invoke(prompt)
    return {"ranked_solutions": response.ranked_solutions}

# Define the mapping logic
def continue_to_evaluation(state: OverallState):
    return [Send("evaluate_solution", {"solution": s}) for s in state["solutions"]]

def continue_to_deep_thought(state: OverallState):
    return [Send("deepen_thought", {"solution": r}) for r in state["reviews"]]

In [5]:
# Construct the graph
graph = StateGraph(OverallState)

graph.add_node("generate_solutions", generate_solutions)
graph.add_node("evaluate_solution", evaluate_solution)
graph.add_node("deepen_thought", deepen_thought)
graph.add_node("rank_solutions", rank_solutions)

graph.add_edge(START, "generate_solutions")
graph.add_conditional_edges("generate_solutions", continue_to_evaluation, ["evaluate_solution"])
graph.add_conditional_edges("evaluate_solution", continue_to_deep_thought, ["deepen_thought"])
graph.add_edge("deepen_thought", "rank_solutions")
graph.add_edge("rank_solutions", END)

app = graph.compile()

In [7]:
import textwrap
from pprint import pprint

# Call the graph
for s in app.stream({
    "input": "improving public transportation in a growing city",
    "perfect_factors": "cost, efficiency, environmental impact, and user experience"
}):
    pprint(s)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


{'generate_solutions': {'solutions': ['Increase frequency of public '
                                      'transportation routes',
                                      'Implement a smart ticketing system with '
                                      'real-time information',
                                      'Invest in electric buses']}}
{'evaluate_solution': {'reviews': ['Increase frequency of public '
                                   'transportation routes']}}
{'evaluate_solution': {'reviews': ['Invest in electric buses']}}


Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


{'evaluate_solution': {'reviews': ['Pros:\n'
                                   '- Improved customer experience: A smart '
                                   'ticketing system with real-time '
                                   'information can provide customers with a '
                                   'more efficient and convenient experience. '
                                   'They can easily track the status of their '
                                   'tickets, get updates on delays, and find '
                                   'answers to their questions without having '
                                   'to wait on hold.\n'
                                   '- Increased efficiency: A smart ticketing '
                                   'system can automate many tasks, such as '
                                   'ticket routing, escalation, and reporting. '
                                   'This can free up staff to focus on more '
                                   '

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


{'deepen_thought': {'deep_thoughts': ['Scenario 1: A customer is experiencing '
                                      'a technical issue with their account. '
                                      'They submit a ticket through the smart '
                                      'ticketing system. The system '
                                      'automatically routes the ticket to the '
                                      'appropriate support team based on the '
                                      "customer's account type and the nature "
                                      'of the issue. The support team can then '
                                      "access the customer's account "
                                      'information and history of interactions '
                                      'to quickly understand the problem and '
                                      'provide a solution.\n'
                                      '\n'
                                   